# Estructuración y organización de datasets

Los datasets utilizados son:

- [20Newsgroups](http://qwone.com/~jason/20Newsgroups/)
- [BAC: The Blog Authorship Corpus](https://huggingface.co/datasets/barilan/blog_authorship_corpus)

Fueron descargados desde el link de Dropbox proporcionado, en el siguiente [enlace](https://huggingface.co/datasets/barilan/blog_authorship_corpus).

Para el siguiente proceso vamos a suponer la siguiente estructura en el directorio.

```
data
│
└───20news-18828
│   └───20news-18828
│       └───alt.atheism
│       └───comp.graphics
│       └───...
│
└───BAC
│   │   5114.male.25.indUnk.Scorpio.xml
│   │   blogs.zip
│
└───EN_Lexicons
│   │   AFINN-111.txt
│   │   senticnet5.py
│   │   SentiWordNet_3.0.0.txt
│   │   WordStat Sentiments.txt
│
└───Multi Domain Sentiment
│   │   negative.review
│   │   processed_acl.tar.gz
│   │   unlabeled.review
```

Nótese que el único paso adicional con respecto a los datos descargados de Dropbox es extraer el archivo `20news-18828.tar.gz`, que da lugar a la carpeta con el mismo nombre.

Primero, importamos algunas librerías de utilidad.

In [10]:
import os, re, zipfile
from typing import List, Tuple

Vamos a definir las rutas para acceder a los textos en `BAC` y `20N`.

In [6]:
twenty_n_path = os.path.join('data', '20news-18828', '20news-18828')
bac_path = os.path.join('data', 'BAC', 'blogs.zip')

Los textos de `20N` se encuentran en un formato que es conveniente limpiar primero con expresiones `regex`. A continuación, definimos una expresión regular para esto.

In [7]:
twenty_n_regex = r'''
^From:.*?\n|                      # Ignora la línea que empieza con 'From:' y lo que sigue hasta el final de la línea
^Subject:.*?\n|                   # Ignora la línea que empieza con 'Subject:' y lo que sigue hasta el final de la línea
^Archive-name:.*?\n|              # Ignora la línea que empieza con 'Archive-name:' y lo que sigue hasta el final de la línea
^Alt-atheism-archive-name:.*?\n|  # Ignora la línea que empieza con 'Alt-atheism-archive-name:' y lo que sigue hasta el final de la línea
^Last-modified:.*?\n|             # Ignora la línea que empieza con 'Last-modified:' y lo que sigue hasta el final de la línea
^Version:.*?\n|                   # Ignora la línea que empieza con 'Version:' y lo que sigue hasta el final de la línea
^.*@.*?\n|                        # Ignora la línea que contiene '@' y lo que sigue hasta el final de la línea
In\sarticle.*?writes:\n|          # Ignora todo lo que está entre 'In article...' y 'writes:'
[^a-zA-Z0-9\s.,]                  # Elimina cualquier carácter que no sea una letra, un número o un espacio
|^>+                              # Elimina '>' al inicio de una línea
|\s*>+                            # Elimina '>' seguido por espacios
^-+$                              # Ignora las líneas que contienen solo '-'
^=+$                              # Ignora las líneas que contienen solo '='
'''

En la siguiente celda, definimos una función para listar los nombres de los directorios de una ruta en particular, y listar de la misma manera los archivos.

In [ ]:
def list_directories(path: str) -> List[str]:
    """
    Retorna una lista de directorios dentro de la ruta especificada.

    Args:
        path (str): Ruta en la que buscar subdirectorios.

    Returns:
        List[str]: Lista de nombres de los subdirectorios.
    """
    return [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]

def list_files(path: str) -> List[str]:
    """
    Retorna una lista de nombres de archivos dentro de la ruta especificada.

    Args:
        path (str): Ruta en la que buscar archivos.

    Returns:
        List[str]: Lista de nombres de archivos
    """
    return [name for name in os.listdir(path) if os.path.isfile(os.path.join(path, name))]

Las siguientes funciones obtienen los textos relevantes de `20N` y `BAC`. Nótese que reciben un archivo o la ruta a este, y extraen el texto que nos interesa.

In [ ]:
def get_relevant_text_twenty_news(path: str) -> str:
    """
    Extrae y limpia el texto relevante de un archivo utilizando regex.

    Args:
        path (str): La ruta del archivo a leer y procesar.

    Returns:
        str: El texto limpio.
    """
    global twenty_n_regex # Usar una expresión regular global predefinida

    with open(path, 'r', encoding='utf-8') as file:
        text = file.read()
        # Limpiar el texto utilizando regex
        cleaned_text = re.sub(twenty_n_regex, '', text, flags=re.VERBOSE | re.MULTILINE)

    # Eliminar espacios en blanco repetidos y retornar el texto
    return ' '.join(cleaned_text.split())

def get_relevant_text_bac(content: str) -> List[Tuple[str, str]]:
    """
    Extrae y limpia el texto relevante de datos BAC, incluyendo fechas y el texto en cuestión (las publicaciones).

    Args:
        content (str): El contenido del texto a procesar.

    Returns:
        List[Tuple[str, str]]: Una lista de tuplas donde cada tupla contiene una fecha y su texto.
    """
    # Extraer todas las fechas y publicaciones del contenido
    dates = re.findall(r'<date>(.*?)</date>', content, re.DOTALL)
    posts = re.findall(r'<post>(.*?)</post>', content, re.DOTALL)
    
    # Limpiar los textos eliminando espacios innecesarios y palabras irrelevantes
    cleaned_posts = [' '.join(re.sub(r'\s+', ' ', post).split()).replace('urlLink', '') for post in posts]
    
    # Combinar las fechas y los textos en tuplas y retornarlas
    return list(zip(dates, cleaned_posts))

Las siguientes son las funciones de procesamiento principales para consolidar todos los archivos tanto de `BAC` como de `20N` en sus propios `txt` aparte. Nótese que generan archivos llamados `consolidated_bac.txt` y `consolidated_news.txt` en el mismo directorio en donde se ejecute este notebook.

In [8]:
def process_twenty_news_data(path: str) -> None:
    """
    Procesa todos los datos de 20N, consolida el texto y lo escribe en un archivo.

    Args:
        ruta (str): La ruta del directorio que contiene los datos de noticias.
    """
    with open('consolidated_news.txt', 'w', encoding='utf-8') as output_file:

        # Listar los subdirectorios que contienen las noticias
        news_dirs = list_directories(path)
        
        # Para cada subdirectorio
        for each_news_dir in news_dirs:
            each_path = os.path.join(path, each_news_dir) # Ruta completa del subdirectorio
            news_files = list_files(each_path) # Listar archivos dentro del subdirectorio
            
            # Para cada archivo de noticias
            for each_news_file in news_files:
                file_path = os.path.join(each_path, each_news_file) # Ruta completa del archivo
                text = get_relevant_text_twenty_news(file_path) # Extraer el texto relevante
                output_file.write(text + '\n') # Escribir el texto en el archivo de salida

def process_bac_data(zip_path: str) -> None:
    """
    Procesa todos los datos de BAC desde un archivo zip, extrae información relevante y la escribe en un archivo.

    Args:
        ruta_zip (str): La ruta al archivo zip que contiene los datos de BAC.
    """
    # Abrir el archivo zip para lectura
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_names = zip_ref.namelist() # Obtener la lista de nombres de archivos en el zip
        
        with open('consolidated_bac.txt', 'w', encoding='utf-8') as output_file:
            # Para cada archivo en el zip
            for file_name in file_names:
                if not file_name.endswith('/'): # Ignorar directorios dentro del zip
                    with zip_ref.open(file_name) as file:
                        try:
                            # Intentar leer el contenido del archivo como UTF-8
                            content = file.read().decode('utf-8')
                        except UnicodeDecodeError:
                            # Si falla, leerlo como latin1
                            content = file.read().decode('latin1')
                        
                        # Extraer las parejas de fecha y texto
                        data_pairs = get_relevant_text_bac(content)
                        
                        # Escribir cada pareja en el archivo de salida
                        for date, post in data_pairs:
                            output_file.write(f"{date} - {post}\n")

Finalmente, la siguiente celda ejecuta estos procesos y genera los archivos consolidados.

In [9]:
process_twenty_news_data(twenty_n_path)
process_bac_data(bac_path)